In [7]:
from IntradayStockPriceCrawler import StockPriceCrawler
import pandas as pd

stock_bank = pd.read_csv('companylist_bank.csv').Symbol
stock_it = pd.read_csv('companylist_it.csv').Symbol

In [4]:
stock_data = pd.DataFrame()
for ticker in stock_bank:
    stock_data_temp = StockPriceCrawler(ticker)
    stock_data_temp.loc[:, 'Ticker'] = ticker
    stock_data = pd.concat([stock_data, stock_data_temp])

In [8]:
stock_data.to_csv('5_min_bank.csv')
stock_data_bank = stock_data.copy()

In [14]:
stock_data_it = pd.DataFrame()
for ticker in stock_it:
    stock_data_temp = StockPriceCrawler(ticker)
    stock_data_temp.loc[:, 'Ticker'] = ticker
    stock_data_it = pd.concat([stock_data_it, stock_data_temp])

In [15]:
stock_data_it.to_csv('5_min_it.csv')

In [18]:
stock_data_bank.loc[:, 'avg_price'] = (stock_data_bank.loc[:, 'High'] + stock_data_bank.loc[:, 'Low'] + stock_data_bank.loc[:, 'Close'])/3
stock_data_bank.head()

,Open,High,Low,Close,Volume,Ticker,avg_price
Date,,,,,,,
2017-01-17 09:30:00,84.8700,84.93,84.81,84.930,4807.0,JPM,84.890000
2017-01-17 09:35:00,85.0700,85.10,84.62,84.870,2351786.0,JPM,84.863333
2017-01-17 09:40:00,85.0600,85.22,84.92,85.080,963014.0,JPM,85.073333
2017-01-17 09:45:00,84.7957,85.08,84.75,85.065,642524.0,JPM,84.965000
2017-01-17 09:50:00,84.4860,84.80,84.36,84.795,768775.0,JPM,84.651667


In [17]:
import numpy as np
def arithmetic_return_action(data, k = 30, p = 0.05, t = 0.15):
    '''
    This function will calculate arithmetic return of k days and based on the sum return to determine actions
    
    :param df: stock data
    :param k: number of days
    :param p: cutoff rate
    :param t: sum of the return rate to  a action: buy, hold, or sell
    :return: stock data labeled with actions
    '''
    df = data.copy()
    df.loc[:, 'temp_return'] = 0
    df.loc[:, 'sum_return']  = 0
    
    
    for i in range(1, k + 1):
        
        df.loc[:, 'temp_return'] = (df.loc[:, 'avg_price'].shift(-i) - df.loc[:, 'Close']) / df.loc[:, 'Close']
        df.loc[np.fabs(df.temp_return) > p, 'sum_return'] += df.loc[(np.fabs(df.temp_return) > p), 'temp_return']

            
    df.loc[df.sum_return >= t, 'action'] = 'buy'    
    df.loc[(df.sum_return < t) & (df.sum_return > -t), 'action'] = 'hold'  
    df.loc[df.sum_return <= -t, 'action'] = 'sell'
    
    return df.loc[:, ['Open', 'High', 'Low', 'Close', 'Volume', 'avg_price', 'sum_return', 'action']]